In [1]:
import warnings
warnings.filterwarnings('ignore',category = DeprecationWarning)
warnings.filterwarnings('ignore',category = UserWarning)
warnings.filterwarnings('ignore',category = FutureWarning)

In [2]:
import os
import time
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
from string import punctuation
import nltk
from nltk.corpus import stopwords
import stop_words
import spacy
import gensim
import gensim.utils
import gensim.corpora as corpora
from gensim import models
import pyLDAvis
import pyLDAvis.gensim

In [3]:
%%time
nlp = spacy.load('en')

C:\ProgramData\Anaconda3\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
C:\ProgramData\Anaconda3\lib\site-packages\msgpack_numpy.py:88: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]
C:\ProgramData\Anaconda3\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
C:\ProgramData\Anaconda3\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
C:\ProgramData\Anaconda3\lib\site-pa

C:\ProgramData\Anaconda3\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
C:\ProgramData\Anaconda3\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
C:\ProgramData\Anaconda3\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
C:\ProgramData\Anaconda3\lib\site-packages\msgpack_numpy.py:88: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]
C:\ProgramData\Anaconda3\lib\site-pa

C:\ProgramData\Anaconda3\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
C:\ProgramData\Anaconda3\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
C:\ProgramData\Anaconda3\lib\site-packages\msgpack_numpy.py:88: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]
C:\ProgramData\Anaconda3\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
C:\ProgramData\Anaconda3\lib\site-pa

C:\ProgramData\Anaconda3\lib\site-packages\msgpack_numpy.py:88: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]
C:\ProgramData\Anaconda3\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
C:\ProgramData\Anaconda3\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
C:\ProgramData\Anaconda3\lib\site-packages\msgpack_numpy.py:88: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]
C:\ProgramData\Anaconda3\lib\site-packages\msgpack_numpy

Wall time: 15.7 s


In [4]:
df = pd.read_csv(r'.\Desktop\train_.csv', encoding = 'latin1')

In [5]:
df['soundBites_with_hashtags'][14]

'apple begins hiring first belgian retail store brussels [mac blog] #aapl'

In [6]:
df.drop(df[df['soundBites_with_hashtags'].isna() == True].index.values,inplace = True)

In [97]:
#corp = list(itertools.chain(*list(df['soundBites'])))

In [8]:
df['soundBites_with_hashtags'] = df['soundBites_with_hashtags'].apply(lambda row: row.split(' '))

In [247]:
#Removing Stopwords
stopword = set(stopwords.words('english'))
stopword_update = [item for item in stop_words.get_stop_words('en') if item not in nltk.corpus.stopwords.words('english')]
stop_list = set(['#coffee','#nescafe','$','nescafe','nestle','nescaf','instant','#coffelove','#coffeelover','#coffeetime','#coffeeholic','#coffeelovers','#instacoffee',
                '#coffeelife','#coffeeaddict','#cafeineaddict','#caffeineaddict','crown','#coffeeforlife','brand','brands','photography','coffee',
                '#coldcoffee','readytodrink','says','said','disha','nestle','colgate','pepsi','#foodforlife','#foodshare','#foodstyling',
                '#photography','sunsilk','dove','north','america','cocacola','footprint','jocky','jockey','ad','#blackcoffee','#morningcoffee',
                'apple','levi\'s','rayban','#fooddiary','#coffeeart','#foodforthought','#coffeeart','#foodbaby','#nescaf','platform','time',
                'indomie','amazon','maker','sales','jar','maker','nonfoil','gives','#foodpost','#foodnetwork','#ccd','market','#feedfeed',
                '#foodtruck','#tastytuesday','taste','shop','network','#caffeine','day','agreement','fashionista','girl','#platter',
                '#coffeemug','#coffeebreak','#cafe','#coffeecup','#coffeelove','#coffeeshop','#coffeegram','good','#mohali','#dreams','#iphonex',
                 '#fitnessjourney','#letsplay','#tennispro','#fashion','#followyoursport','#tennisday','#coffeelove','#gareebonkarobinhood','#nestl',
                'bath','#morning','feedfeed','lux','reached','lower','singh','coke','two','knorr','teena','kantar','big','schneider','one','war','sent',
                'bru','reliance','#latte','#starbucks','#nescafegold','#cofferlovers','#coffeefever','free','deal','gram','till','movenpick','sale','coop',
                '#tea','#espresso','#coffeeoftheday','hold','#fashionblogger','#coffeebeans','three','lifebuoy','eden','continued','statement','including',
                '#fashiondesigner','#model','#modeling','offering','dolce','#worldofmarketing','#chandigarh','#gurgaon','#auro','fashionistamodelnescafe',
                '#instapic','allensolly','model','gorgeous','#chef','#greenery','#engineering','#blogger','get','#foodislife','see','msrp','go','nitdgp',
                '#streetsofbengal','#pmc','#potd','#photo','#goa','#coffee_time','#photoshop','#goodmorning','followsharecomment','rights'])
stopword.update(stop_list),
stopword.update(set(stopword_update))
stopword.update(set(string.ascii_lowercase))

In [248]:
df["soundBites_with_hashtags"] = df["soundBites_with_hashtags"].apply(lambda row: [item for item in row if item not in stopword])

In [9]:
dictionary = corpora.Dictionary(df['soundBites_with_hashtags'])

In [10]:
corpus = [dictionary.doc2bow(text) for text in df['soundBites_with_hashtags']]

In [11]:
model = gensim.models.LdaMallet( corpus=corpus, num_topics=20, id2word=dictionary)

AttributeError: module 'gensim.models' has no attribute 'LdaMallet'

In [251]:
lda_model = models.LdaModel(corpus=corpus, num_topics=10, id2word=dictionary,)

In [252]:
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)

In [253]:
lda_model.show_topics()

[(0,
  '0.004*"classic" + 0.004*"price" + 0.004*"starbucks" + 0.003*"gold" + 0.003*"blend" + 0.003*"last" + 0.003*"drink" + 0.003*"nespresso" + 0.003*"packaged" + 0.003*"oldspice"'),
 (1,
  '0.019*"nespresso" + 0.012*"company" + 0.010*"strong" + 0.010*"growth" + 0.010*"owns" + 0.006*"starbucks" + 0.005*"india" + 0.004*"products" + 0.004*"#music" + 0.004*"jab"'),
 (2,
  '0.005*"drink" + 0.004*"cup" + 0.004*"eat" + 0.004*"shirt" + 0.004*"nespresso" + 0.004*"sizzling" + 0.004*"underwear" + 0.004*"gillette" + 0.004*"pant" + 0.004*"shave"'),
 (3,
  '0.007*"nespresso" + 0.006*"first" + 0.005*"company" + 0.004*"last" + 0.004*"harvest" + 0.004*"home" + 0.004*"buy" + 0.003*"starbucks" + 0.003*"executive" + 0.003*"chief"'),
 (4,
  '0.007*"nespresso" + 0.005*"cold" + 0.005*"world" + 0.004*"growth" + 0.004*"company" + 0.004*"many" + 0.003*"owns" + 0.003*"strong" + 0.003*"made" + 0.003*"drink"'),
 (5,
  '0.003*"cold" + 0.002*"world" + 0.002*"products" + 0.002*"flagship" + 0.002*"far" + 0.002*"risen

In [254]:
pyLDAvis.display(vis)

In [255]:
topn_words = {'Topic_' + str(i): [word for word, prob in lda_model.show_topic(i, topn=10)] for i in range(0, lda_model.num_topics)}

In [256]:
pd.DataFrame(topn_words).to_csv('./Desktop/Stream_LDA.csv')